### below is the final script for GI Commission

In [15]:
# standard import
from datetime import date
import datetime
import pandas as pd
import numpy as np
import os
import glob
import PyPDF2
import docx

import re

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

import warnings
warnings.filterwarnings("ignore")

folder_path = input("Please input the directory to the files -> /Users/jiayikoh/Downloads/07 GI Commission 3/\n")

while os.path.isdir(folder_path) != True:
    folder_path = input("Directory does not exist, please re-enter the directory to the files -> /Users/jiayikoh/Downloads/07 GI Commission 3/\n")
#os.chdir(folder_path)
os.chdir('/Users/jiayikoh/KJY/IPP/GI_Commission_20240805')

# state the Comm Mth.
input_comm_mth = input("Please input the Comm Mth -> '2024-02-01'\n")
comm_mth = datetime.datetime.strptime(input_comm_mth, "%Y-%m-%d").date()

# Read all sheets into a dictionary of dataframes
pattern='C1A'
file_pattern = pattern + '*.xls*' 
matching_files = glob.glob(os.path.join(folder_path, file_pattern))
uob_cb_dict = pd.read_excel(matching_files[0], sheet_name=None, header=4)

# Concatenate all dataframes into one
uob_cb = pd.concat(uob_cb_dict.values(), ignore_index=True)

# Read all sheets into a dictionary of dataframes
pattern='C2A'
file_pattern = pattern + '*.xls*' 
matching_files = glob.glob(os.path.join(folder_path, file_pattern))

rhb_cb_dict = pd.read_excel(matching_files[0], sheet_name=None, header=4)

# Concatenate all dataframes into one
rhb_cb = pd.concat(rhb_cb_dict.values(), ignore_index=True)


# extract the insurer name
uob_cb['Extracted_Insurer'] = uob_cb['Name'].str.extract('([^\s]+)', expand=False).str.lower()
rhb_cb['Extracted_Insurer'] = rhb_cb['Name'].str.extract('([^\s]+)', expand=False).str.lower()

# concatenate both cashbook df
all_cb = pd.concat([uob_cb, rhb_cb], axis=0)

# to load the current P0 working file to get the previous information for the missing TFAR in some of the insurer's files
pattern='P0'
file_pattern = pattern + '*.xls*' 
matching_files = glob.glob(os.path.join(folder_path, file_pattern))
wk_dict = pd.read_excel(matching_files[0], sheet_name=None)

# define the keys to be concatenate
paid_keys = [key for key in wk_dict.keys() if key.startswith('PAID')]
unpaid_keys = [key for key in wk_dict.keys() if key.startswith('UNPAID')]

# concatenate both
all_keys = paid_keys + unpaid_keys

# concatenate the corresponding dataframe
previous_wk = pd.concat([wk_dict[key] for key in all_keys]).drop_duplicates()

from gi_test import GI_commission

from Name_matching import Name_matching

gi = GI_commission(folder_path=folder_path, all_cb=all_cb, previous_wk=previous_wk)

# concatenate all the df
wk_df_1 = pd.concat([gi.run_aig_1('01_01A'), #aig
                     gi.run_aig_1('01_01B'),
                     gi.run_aig_1('01_01C'),
                     gi.run_aig_1('01_01D'),
                     gi.run_aig_1('01_01E'),
           gi.run_aia_1('03_01A'), #aia
           gi.run_aia_1('03_01B'),
           gi.run_aia_1('03_01C'),
           gi.run_aia_1('03_01D'),
           gi.run_aia_1('03_01E'),
           gi.run_allianz_1('04_01A'), #allianz
           gi.run_allianz_1('04_01B'),
           gi.run_allianz_1('04_01C'),
           gi.run_allianz_1('04_01D'),
           gi.run_allianz_1('04_01E'),
           gi.run_allied_1('06_01A'), #allied
           gi.run_allied_1('06_01B'),
           gi.run_allied_1('06_01C'),
           gi.run_allied_1('06_01D'),
           gi.run_allied_1('06_01E'),
           gi.run_allied_2('06_02A'),
           gi.run_allied_2('06_02B'),
           gi.run_allied_2('06_02C'),
           gi.run_allied_2('06_02D'),
           gi.run_allied_2('06_02E'),
           gi.run_chubb_1('09_01A'), #chubb
           gi.run_chubb_1('09_01B'),
           gi.run_chubb_1('09_01C'),
           gi.run_chubb_1('09_01D'),
           gi.run_chubb_1('09_01E'),
           gi.run_chubb_2('09_02A'),
           gi.run_chubb_2('09_02B'),
           gi.run_chubb_2('09_02C'),
           gi.run_chubb_2('09_02D'),
           gi.run_chubb_2('09_02E'),
           gi.run_chubb_3('09_03A'),
           gi.run_chubb_3('09_03B'),
           gi.run_chubb_3('09_03C'),
           gi.run_chubb_3('09_03D'),
           gi.run_chubb_3('09_03E'),
           gi.run_delta_1('11_01A'), #delta
           gi.run_delta_1('11_01B'),
           gi.run_delta_1('11_01C'),
           gi.run_delta_1('11_01D'),
           gi.run_delta_1('11_01E'),
           gi.run_ergo_1('13_01A'), #ergo
           gi.run_ergo_1('13_01B'),
           gi.run_ergo_1('13_01C'),
           gi.run_ergo_1('13_01D'),
           gi.run_ergo_1('13_01E'),
           gi.run_fwd_1('15_01A'), #fwd
           gi.run_fwd_1('15_01B'),
           gi.run_fwd_1('15_01C'),
           gi.run_fwd_1('15_01D'),
           gi.run_fwd_1('15_01E'),
           gi.run_hla_1('19_01A'), #hla
           gi.run_hla_1('19_01B'),
           gi.run_hla_1('19_01C'),
           gi.run_hla_1('19_01D'),
           gi.run_hla_1('19_01E'),
           gi.run_hsbc_1('20_01A'), #hsbc
           gi.run_hsbc_1('20_01B'),
           gi.run_hsbc_1('20_01C'),
           gi.run_hsbc_1('20_01D'),
           gi.run_hsbc_1('20_01E'),
           gi.run_hsbc_2('20_02A'),
           gi.run_hsbc_2('20_02B'),
           gi.run_hsbc_2('20_02C'),
           gi.run_hsbc_2('20_02D'),
           gi.run_hsbc_2('20_02E'),
           gi.run_income_1('21_01A'), #income
           gi.run_income_1('21_01B'),
           gi.run_income_1('21_01C'),
           gi.run_income_1('21_01D'),
           gi.run_income_1('21_01E'),
           gi.run_liberty_1('23_01A'), #liberty
           gi.run_liberty_1('23_01B'),
           gi.run_liberty_1('23_01C'),
           gi.run_liberty_1('23_01D'),
           gi.run_liberty_1('23_01E'),
           gi.run_msig_1('25_01A'), #msig
           gi.run_msig_1('25_01B'),
           gi.run_msig_1('25_01C'),
           gi.run_msig_1('25_01D'),
           gi.run_msig_1('25_01E'),
           gi.run_nhi_1('26_01A'), #nhi
           gi.run_nhi_1('26_01B'),
           gi.run_nhi_1('26_01C'),
           gi.run_nhi_1('26_01D'),
           gi.run_nhi_1('26_01E'),
           gi.run_qbe_1('28_01A'), #qbe
           gi.run_qbe_1('28_01B'),
           gi.run_qbe_1('28_01C'),
           gi.run_qbe_1('28_01D'),
           gi.run_qbe_1('28_01E'),
           gi.run_singlife_1('30_01A'), #singlife
           gi.run_singlife_1('30_01B'),
           gi.run_singlife_1('30_01C'),
           gi.run_singlife_1('30_01D'),
           gi.run_singlife_1('30_01E'),
           gi.run_sompo_1('31_01A'), #sompo
           gi.run_sompo_1('31_01B'),
           gi.run_sompo_1('31_01C'),
           gi.run_sompo_1('31_01D'),
           gi.run_sompo_1('31_01E'),
           gi.run_tm_1('32_01A'), #tm
           gi.run_tm_1('32_01B'),
           gi.run_tm_1('32_01C'),
           gi.run_tm_1('32_01D'),
           gi.run_tm_1('32_01E'),
           gi.run_tm_2('33_01A'),
           gi.run_tm_2('33_01B'),
           gi.run_tm_2('33_01C'),
           gi.run_tm_2('33_01D'),
           gi.run_tm_2('33_01E')
           ], ignore_index=True).drop_duplicates()


name_far = Name_matching(folder_path=folder_path, all_advisor_df=wk_df_1)

comm_far = name_far.matching_far()

# get the match score of 100 only
comm_far = comm_far[comm_far['matching_score']==100]

# function to extract the first element after "-"

pattern = r"(?<=-).*.*(?=-)"

def extract_after_or_first(text):
  """Extracts the element after the first "-" or the first word.

  Args:
      text: The string to extract from.

  Returns:
      The first element after the first "-" or the first word, or None if empty.
  """
  try:
    match = re.search(pattern, text)
    if match:
      return match.group()
    else:
      return None
  except:
    return None
  
# this is to extract the first element after "-" and strip off the whitespace at the start & back
wk_df_1['TFAR'] = np.where(wk_df_1['TFAR'].apply(extract_after_or_first) == None, wk_df_1['TFAR'], wk_df_1['TFAR'].apply(extract_after_or_first).str.strip())

# read the GI e-submission list
file_pattern = 'W3*.xls*'

matching_files = glob.glob(os.path.join(folder_path, file_pattern))

# load the e-submission list for the referrer list
e_sub = pd.read_excel(matching_files[0])

# this is to get the latest TFAR & Referrer based on the latest policy no (we will have to match to find the TFAR)
# group by the policy number and then filter the latest file name (which is a datetime)
previous_wk_pol_1 = previous_wk.sort_values(by=['File Name'], ascending=False).groupby(by=['Policy No'])

# to get the first one which is the latest information
previous_wk_pol_2 = previous_wk_pol_1.head(1)

# this is to get the latest TFAR & Referrer based on the latest Insured Name (we will have to match to find the TFAR)
# group by the Insured Name and then filter the latest file name (which is a datetime)
previous_wk_ins_1 = previous_wk.sort_values(by=['File Name'], ascending=False).groupby(by=['Insured '])

# to get the first one which is the latest information
previous_wk_ins_2 = previous_wk_ins_1.head(1)

# 1st: merge with e_sub using policy number to get Referrer name and TFAR Name
wk_df_1_merge_1 = pd.merge(wk_df_1, e_sub[['Policy No', 'Name of TFAR', 'Name of Referral']].rename(columns={'Name of TFAR':'TFAR_esub', 'Name of Referral':'Referral_esub'}), how='left', left_on='Policy no.', right_on='Policy No')

# 2nd: merge with previous_wk using policy number to get Referral and TFAR
wk_df_1_merge_2 = pd.merge(wk_df_1_merge_1, previous_wk_pol_2[['Policy No', 'TFAR', 'REFFERAL ', 'Insured ']].rename(columns={'TFAR':'TFAR_wk_pol', 'REFFERAL ':'Referral_wk_pol', 'Insured ':'Insured_wk_pol'}), how='left', left_on='Policy no.', right_on='Policy No')

# 3rd: merge with previous_wk using insured name to get Referral and TFAR
wk_df_1_merge_3 = pd.merge(wk_df_1_merge_2, previous_wk_ins_2[['Insured ', 'TFAR', 'REFFERAL ']].rename(columns={'TFAR':'TFAR_wk_ins', 'REFFERAL ':'Referral_wk_ins'}), how='left', left_on=wk_df_1_merge_2['Insured Name'].str.lower(), right_on=previous_wk_ins_2['Insured '].str.lower())

df_filled_1 = wk_df_1_merge_3[['TFAR', 'TFAR_wk_pol', 'TFAR_wk_ins', 'TFAR_esub']].rename(columns={'TFAR':'CommState_TFAR', 'TFAR_wk_pol':'PreviousWork_TFAR_pol', 
                                                                                                   'TFAR_wk_ins':'PreviousWork_TFAR_ins', 'TFAR_esub':'esub_TFAR'})

# fill missing values with column-wise forward fill
df_filled_2 = df_filled_1.fillna(axis=1, method='ffill')

def get_all_values_and_first_available(row):
  """
  Safely retrieves the first available value in a row, handling cases with all NaN values.

  Args:
      row: A pandas Series representing a row from the DataFrame.

  Returns:
      The first non-NaN value in the row, or None if all values are NaN.
  """
  try:
    # get all values including NaN using tolist()
    all_values = row.tolist()
    
    # get the first available value
    first_available = row.dropna().iloc[0]
    
    # check if any value is nan
    has_nan = row.isnull().any()
    # Attempt to get the first non-NaN value using iloc[0]
    return all_values, first_available, has_nan
  except IndexError:
    # Handle cases where all values are NaN (IndexError due to empty Series)
    return None
  
def extract_nth_element(data, nth):
  """Extracts the second element from the tuple, handling None values.

  Args:
      data: The tuple (element from the Series).

  Returns:
      The second element from the tuple, or None if the data is None or IndexError occurs.
  """
  try:
    return data[nth]
  except (IndexError, TypeError):
    return None
  
# extract all TFAR value from different source
df_filled_2['all_TFARs'] = df_filled_2.apply(get_all_values_and_first_available, axis=1).apply(lambda x: extract_nth_element(x, 0))

# take the first available value
# df_filled_2['first_available'] = df_filled_2[['TFAR_wk_pol', 'TFAR_wk_ins', 'TFAR_esub']].apply(get_all_values_and_first_available, axis=1).apply(lambda x: extract_nth_element(x, 1))
df_filled_2['first_available'] = df_filled_2.apply(get_all_values_and_first_available, axis=1).apply(lambda x: extract_nth_element(x, 1))

# concatenate the df_filled_2 with the main df
wk_df_1_merge_4 = pd.concat([wk_df_1_merge_3, df_filled_2], axis=1)

# rule: if TFAR is nan, take the first available
wk_df_1_merge_4['Final_TFAR'] = np.where(wk_df_1_merge_4['TFAR'].isna(), wk_df_1_merge_4['first_available'], wk_df_1_merge_4['TFAR'])

# rule: if insured is nan, take from the previous working file
wk_df_1_merge_4['Final_Insured'] = np.where(wk_df_1_merge_4['Insured Name'].isna(), wk_df_1_merge_4['Insured_wk_pol'], wk_df_1_merge_4['Insured Name'])

wk_df_1 = wk_df_1_merge_4.copy()

# output another data featuring the TFAR from different source

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(f'{folder_path}/Working_file_check_tfar_{date.today():%Y%m%d}.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
wk_df_1_merge_4[['Policy no.', 'CommState_TFAR', 'PreviousWork_TFAR_pol', 'PreviousWork_TFAR_ins', 'esub_TFAR', 'first_available']].drop_duplicates().to_excel(writer)

writer.close()

wk_df_1['TFAR'] = wk_df_1['Final_TFAR']
wk_df_1['Insured Name'] = wk_df_1['Final_Insured']

# read the FAR masterlist
file_pattern = 'W1*.xls*' 

matching_files = glob.glob(os.path.join(folder_path, file_pattern))

far_master = pd.read_excel(matching_files[0], skiprows=1)


# read the GI e-submission list
file_pattern = 'W3*.xls*'

matching_files = glob.glob(os.path.join(folder_path, file_pattern))

# load the e-submission list for the referrer list
e_sub = pd.read_excel(matching_files[0])

# merge with itself to get the manager's advisory group
far_master_1 = pd.merge(far_master, far_master[['NEW FAR NAME', 'Advisory Group']].rename(columns={'NEW FAR NAME':'Manager Name', 'Advisory Group':'Manager AG'}), 
                      how='left', left_on='Manager', right_on='Manager Name').drop_duplicates()

# merge with itself to get the old agent's manager details
far_master_2 = pd.merge(far_master_1, far_master[['NEW FAR NAME', 'Manager', 'Advisory Group', 'Mgr Rate']].rename(columns={'NEW FAR NAME':'Old_FAR', 'Manager':'Manager 2 Name', 'Advisory Group':'Manager 2 AG', 'Mgr Rate':'Manager 2 GR%'}),
                     how='left', left_on='FAR (Old)', right_on='Old_FAR').rename(columns={'Mgr Rate':'Manager GR%'}).drop_duplicates()


# will have to remove the "all_TFARs" column as it contains lists -> will not be able to do .drop_duplicates()
del wk_df_1['all_TFARs']
del wk_df_1['Policy No_x']

# find the name in FAR masterlist using the revised name list provided by the name matching algorithm
wk_df_2 = pd.merge(wk_df_1, comm_far[['ADVISER', 'matched_name']], how='left', left_on='TFAR', right_on='ADVISER')

# match the adviser details using the name matching list
wk_df_3 = pd.merge(wk_df_2, far_master_2[['NEW FAR NAME', 'Advisory Group', 2024, 'FAR Status', 'NRIC', 'Manager Name', 'Manager AG', 'Manager GR%', 'Manager 2 Name', 'Manager 2 AG', 'Manager 2 GR%']].rename(columns={2024:'GR%', 'NRIC':'FAR_NRIC'}), how='left', left_on='matched_name', right_on='NEW FAR NAME')

# match policy number to get referrer details
wk_df_4 = pd.merge(wk_df_3, e_sub[['Policy No', 'Name of Referral']].rename(columns={'Name of Referral':'Referrer'}), how='left', left_on='Policy no.', right_on='Policy No').drop_duplicates()

# remove Not Applicable in Referrer
wk_df_4['Referrer'] = np.where(wk_df_4['Referrer'] == 'Not Applicable', np.NaN, wk_df_4['Referrer'])

# find the name in FAR masterlist using the revised name list provided by the name matching algorithm
wk_df_5 = pd.merge(wk_df_4, comm_far[['ADVISER', 'matched_name']].rename(columns={'matched_name':'referrer_matched_name'}), how='left', left_on='Referrer', right_on='ADVISER')


# label GST Type
insurer_list = list(wk_df_5['Insurer'].unique())

new_df = []

for i, j in enumerate(insurer_list):
    df_working = wk_df_5[wk_df_5['Insurer'] == j]
    if j == 'AIG-GI':
        df_working['GST Type'] = np.where(df_working['GST amt'] == 0, 'Z', 'V')
        new_df.append(df_working)
    elif j == 'AIA-GI':
        df_working['GST Type'] = np.where(df_working['GST amt'] == 0, 'Z', 'G')
        new_df.append(df_working)
    elif j == 'ALLIANZ-GI':
        df_working['GST Type'] = np.where(df_working['GST amt'] == 0, 'Z', 'G')
        new_df.append(df_working)
    elif j == 'ALLIED-GI':
        df_working['GST Type'] = np.where(df_working['GST amt'] == 0, 'Z', 'G')
        new_df.append(df_working)
    elif j == 'CHUBB-GI':
        df_working['GST Type'] = np.where(df_working['GST amt'] == 0, 'Z', 'G')
        new_df.append(df_working)
    elif j == 'DELTA-GI':
        df_working['GST Type'] = np.where(df_working['GST amt'] == 0, 'Z', 'G')
        new_df.append(df_working)
    elif j == 'FWD-GI':
        df_working['GST Type'] = np.where(df_working['$txn gst commission'] == 0, 'Z', 'G')
        new_df.append(df_working)   
    elif j == 'HLA-GI':
        df_working['GST Type'] = np.where(df_working['GST amt'] == 0, 'Z', 'G')
        new_df.append(df_working)
    elif j == 'GE-GI':
        df_working['GST Type'] = np.where(df_working['GST amt'] == 0, 'Z', 'G')
        new_df.append(df_working)
    elif j == 'HSBC LIFE-GI':
        df_working['GST Type'] = 'V'
        new_df.append(df_working)
    elif j == 'INCOME-GI':
        df_working['GST Type'] = 'Z'
        new_df.append(df_working)
    elif j == 'INDIA-GI':
        df_working['GST Type'] = np.where(df_working['GST amt'] == 0, 'Z', 'G')
        new_df.append(df_working)
    elif j == 'LIBERTY-GI':
        df_working['GST Type'] = 'V'
        new_df.append(df_working)
    elif j == 'MSIG-GI':
        df_working['GST Type'] = np.where(df_working['GST amt'] == 0, 'Z', 'G')
        new_df.append(df_working)
    elif j == 'NHI-GI':
        df_working['GST Type'] = np.where(df_working['GST amt'] == 0, 'Z', 'G')
        new_df.append(df_working)
    elif j == 'QBE-GI':
        df_working['GST Type'] = np.where(df_working['GST amt'] == 0, 'Z', 'G')
        new_df.append(df_working)
    elif j == 'SINGLIFE-GI':
        df_working['GST Type'] = np.where(df_working['GST amt'] == 0, 'Z', 'G')
        new_df.append(df_working)
    elif j == 'SOMPO-GI':
        df_working['GST Type'] = np.where(df_working['GST amt'] == 0, 'Z', 'V')
        new_df.append(df_working)
    elif j == 'TM-GI':
        df_working['GST Type'] = 'G'
        new_df.append(df_working)
    elif j == 'TM LIFE-GI':
        df_working['GST Type'] = 'G'
        new_df.append(df_working)
        
        
wk_df_6 = pd.concat(new_df)
    

# make a soft copy
wk_df = wk_df_6.drop_duplicates().copy()

# delete TFAR column
del wk_df['TFAR']
del wk_df['Referrer']

# insert a column for Comm Mth.
wk_df['Comm Mth.'] = comm_mth

# get the required fields
columns = ['Comm Mth.', 'Cashbook ref. no.', 'Policy no.', 'Insured Name', 'Insurer', 'TFAR', 'Referrer', 'Advisory Group', 'Comm.Recd (with GST)', 'GST Type', 'GR%', 'Pol Date', 'FAR Receiving Comm Status', 'FAR_NRIC', 'Manager Name', 'Manager AG', 'Manager GR%', 'Manager 2 Name', 'Manager 2 AG', 'Manager 2 GR%']

wk_df = wk_df.rename(columns={'Advisory Group_y':'Advisory Group', 'matched_name':'TFAR', 'referrer_matched_name':'Referrer', 'GR%_y':'GR%', 'FAR Status':'FAR Receiving Comm Status'})[columns].drop_duplicates()

# below function extract the first two digits after the alphabets for the cashbook year
def extract_first_two_digits(text):
  if pd.isna(text):
    return np.nan  # Return NaN if the value is NaN
  else:
    match = re.search(r'\d{2}', text)
    return match.group() if match else np.nan  # Return NaN if no match found

wk_df['Cashbook ref. no.'] = wk_df['Cashbook ref. no.'] + ' (Yr 20' + wk_df['Cashbook ref. no.'].apply(extract_first_two_digits) + ')'

# title case for Insured Name and Advisory Group
#wk_df['Insured Name'] = wk_df['Insured Name'].str.title()
#wk_df['Advisory Group'] = wk_df['Advisory Group'].str.title()

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(f'{folder_path}/S1 - Working_file_{date.today():%Y%m%d}.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
wk_df.to_excel(writer, sheet_name='Data')
#df_1_pivot.to_excel(writer, sheet_name='Pivot')

# Close the Pandas Excel writer and output the Excel file.
writer.close()